## This jupyter Notebook is for the first home-work in the MLOPS Zoomcamp of 2023 Cohort

In [2]:
import pandas as pd
import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error
import math

## First Question 
### We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

- Download the data for January and February 2022. 
* We will use the February data as Validation set

- Read the data for January. How many columns are there?


In [6]:
df = pd.read_parquet('./data/yellow_tripdata_2022-01.parquet')
df.shape[1]

19

## Second Question
 - Now let's compute the duration variable. It should contain the duration of a ride in minutes.

 - What's the standard deviation of the trips duration in January?



In [8]:

old_length = len(df) # we will use this value for later question
df['Duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime 
df['Duration'] = df.Duration.apply(lambda td: td.total_seconds() /60)
df.Duration.std()

46.44530513776802

## Third Question
 - Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

 - What fraction of the records left after you dropped the outliers?


#### WE can use to options to find percentage of records after we dropped the outliers ( rides that took less than or equal to an hour and greater or eqaul than a minute
* -> we can find the mean or we can divide the length of the dataframe after we dropped the outliers by thier lenght before 

In [9]:
((df.Duration >= 1) & (df.Duration <=60)).mean()

0.9827547930522406

In [19]:

df = df[(df.Duration >= 1) & (df.Duration <=60)]
new_length = len(df)
(new_length / old_length)* 100


98.27547930522405

In [31]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
df[categorical] = df[categorical].astype(str)

In [33]:
#df.Duration.describe(percentiles=[0.95, 0.98, 0.99])

In [38]:
train_dict = df[categorical + numerical].to_dict(orient = 'records')

dv = DictVectorizer()
x_train = dv.fit_transform(train_dict)

target = 'Duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(x_train, y_train)

y_pred = lr.predict(x_train)
mean_squared_error(y_train, y_pred, squared=False)


7.001496179375642

## Fourth Question
- Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries
- Fit a dictionary vectorizer
- Get a feature matrix from it
- What's the dimensionality of this matrix (number of columns)?


In [35]:
# because we added additional columns above( Duration column) I have to deduct 1
x_train.shape[1] -1 

515

## Fifth Question
- Train a plain linear regression model with default parameters
- Calculate the RMSE of the model on the training data
- What's the RMSE on train?

In [37]:
mean_squared_error(y_train, y_pred, squared=False)


7.001496179375642

In [39]:
# sns.distplot(y_pred, label = 'prediction')
# sns.distplot(y_train, label = 'actual')

# plt.legend()

In [40]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['Duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime 
    df['Duration'] = df.Duration.apply(lambda td: td.total_seconds() /60)

    df = df[(df.Duration >= 1) & (df.Duration <=60)]
    
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [41]:
df_train = read_dataframe('./data/yellow_tripdata_2022-01.parquet')
df_val = read_dataframe('./data/yellow_tripdata_2022-02.parquet')

In [42]:
len(df_train), len(df_val)

(2421440, 2918187)

In [43]:
df_train['PU_DO'] = df_train['PULocationID'] +'_'+ df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] +'_'+ df_val['DOLocationID']

In [44]:
categorical =['PULocationID', 'DOLocationID'] #['PU_DO']
numerical = ['trip_distance']


dv = DictVectorizer()

train_dict = df_train[categorical + numerical].to_dict(orient = 'records')
x_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient = 'records')
x_val = dv.transform(val_dict)



In [45]:
target = 'Duration'

y_train = df_train[target].values
y_val = df_val[target].values


## Sixth Question

- What's the RMSE on validation?


In [47]:
lr = LinearRegression()
lr.fit(x_train, y_train)

y_pred = lr.predict(x_val)

mean_squared_error(y_val, y_pred, squared=False)


7.795498157775055

### if we had used Ridge Regression the RMSE would have been a little bit higher 
* Linear Regression gives as a better ( lower) value

In [48]:
lr = Ridge()
lr.fit(x_train, y_train)

y_pred = lr.predict(x_val)

mean_squared_error(y_val, y_pred, squared=False)


7.8354868868628165